In [2]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, explained_variance_score, mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from gensim.models import Word2Vec

/home/nikita/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
2018-05-15 17:43:46,337 : INFO : 'pattern' package not found; tag filters are not available for English


In [3]:
import time
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten, Activation
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score

In [4]:
data = pd.read_csv('/home/nikita/PycharmProjects/natural_language/data/test_prepocess.csv')
data.head()

,stars,text,useful,funny
0,3,The good: mini doughnuts and americano friendl...,1,0
1,3,I really like the décor of the place and the w...,0,0
2,5,"Great service, clean and great food, this plac...",1,0
3,5,This place has never failed on me! \nLate nigh...,0,0
4,2,This is a fast food spin off from Paramount Fi...,1,1


In [5]:
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)
train_batch = train_data[0:100000]
test_batch = test_data[0:50000]

In [6]:
def build_sentences_vector(sentence):
    words = sentence.split()
    result = np.zeros(100)
    size = 0
    for word in words:
        if word in embeding_model.wv.vocab:
            result+=embeding_model.wv[word]
            size+=1
    if size == 0:
        size = 1
    result /= size
    return result

In [ ]:
def evaluate_prediction(predictions, target, title="Confusion matrix"):
    print('cross_val_score              %s' % accuracy_score(target, predictions))
#     print('explained_variance_score     %s' % explained_variance_score(target, predictions))
#     print('mean_absolute_error          %s' % mean_absolute_error(target, predictions))
#     print('mean_squared_error           %s' % mean_squared_error(target, predictions))
#     print('mean_squared_log_error       %s' % mean_squared_log_error(target, predictions))
#     print('r2_score                     %s' % r2_score(target, predictions))
    print()
    cm = confusion_matrix(target, predictions)
    print('confusion matrix\n %s' % cm)
    print('(row=expected, col=predicted)')
    
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plot_confusion_matrix(cm_normalized, title + ' Normalized')
    
def plot_confusion_matrix(cm, title='Матрица ошибок', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    my_tags = pd.unique(data.stars)
    tick_marks = np.arange(len(my_tags))
    target_names = my_tags
    plt.xticks(tick_marks, target_names, rotation=45)
    plt.yticks(tick_marks, target_names)
    plt.tight_layout()
    plt.ylabel('Исходные метки')
    plt.xlabel('Полученные метки')

def predict(vectorizer, classifier, data):
    data_features = vectorizer.transform(data.text)
    predictions = classifier.predict(data_features)
    target = data.stars
    evaluate_prediction(predictions, target)

In [7]:
embeding_model = Word2Vec.load('Yelp_embedings')
len(embeding_model.wv.vocab)

2018-05-15 17:44:19,317 : INFO : loading Word2Vec object from Yelp_embedings
2018-05-15 17:44:22,990 : INFO : loading wv recursively from Yelp_embedings.wv.* with mmap=None
2018-05-15 17:44:22,991 : INFO : setting ignored attribute vectors_norm to None
2018-05-15 17:44:22,992 : INFO : loading vocabulary recursively from Yelp_embedings.vocabulary.* with mmap=None
2018-05-15 17:44:22,993 : INFO : loading trainables recursively from Yelp_embedings.trainables.* with mmap=None
2018-05-15 17:44:22,993 : INFO : setting ignored attribute cum_table to None
2018-05-15 17:44:22,997 : INFO : loaded Yelp_embedings


101865

In [8]:
%%time
train_data_features = []
for sentense in train_batch.text:
    train_data_features.append(build_sentences_vector(sentense))
test_data_transform = []
for sentense in test_batch.text:
    test_data_transform.append(build_sentences_vector(sentense))

CPU times: user 1min 32s, sys: 477 ms, total: 1min 32s
Wall time: 1min 33s


In [9]:
train_data_features = pd.DataFrame(train_data_features)
test_data_transform = pd.DataFrame(test_data_transform)

In [10]:
%%time
big_X = train_data_features.append(test_data_transform)
scaler = MinMaxScaler()
scaler.fit(big_X)
train_data_features = scaler.transform(train_data_features)
test_data_transform = scaler.transform(test_data_transform)

CPU times: user 189 ms, sys: 200 ms, total: 389 ms
Wall time: 599 ms


In [11]:
train_data_features[0].shape

(100,)

In [12]:
train_data_features[0]

array([0.47768725, 0.50261539, 0.7106637 , 0.60011637, 0.30857185,
       0.46072202, 0.58532382, 0.35489181, 0.56592655, 0.52509615,
       0.34207966, 0.46469938, 0.28336177, 0.48729129, 0.56763504,
       0.48545882, 0.3633329 , 0.50221415, 0.4790287 , 0.44466971,
       0.60395302, 0.21026502, 0.71012634, 0.61785903, 0.32811395,
       0.40180564, 0.42955268, 0.52559043, 0.41086256, 0.41418741,
       0.7674902 , 0.54301586, 0.61783318, 0.58074126, 0.55888046,
       0.54453445, 0.36168717, 0.54021471, 0.45886499, 0.64497301,
       0.68964035, 0.66150094, 0.42380001, 0.29542307, 0.2645298 ,
       0.64970587, 0.59399217, 0.70192113, 0.67110966, 0.66881021,
       0.52171434, 0.47996132, 0.52243193, 0.59537827, 0.46571829,
       0.55339619, 0.73363276, 0.42394804, 0.23819312, 0.43204729,
       0.68764806, 0.66526823, 0.39650339, 0.48692192, 0.21172303,
       0.69001408, 0.35145527, 0.39906459, 0.34272367, 0.48391983,
       0.44863189, 0.6544023 , 0.37294765, 0.48754314, 0.75848

In [13]:
model = Sequential([
    Dense(107, input_dim=100, activation='relu'),
    Dense(100, activation='relu'),
    Dense(6, activation='softmax')
])
epochs = 30
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 107)               10807     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10800     
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 606       
Total params: 22,213
Trainable params: 22,213
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
Y = np_utils.to_categorical(train_batch.stars)
print(Y)

[[0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 ...
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0.]]


In [15]:
print(train_data_features[0].shape, Y[0].shape)

(100,) (6,)


In [16]:
%%time
model.fit(np.array(train_data_features), np.array(Y), verbose=2, epochs=30, validation_split=0.15)

Train on 85000 samples, validate on 15000 samples
Epoch 1/30
 - 5s - loss: 0.1164 - acc: 0.4320 - val_loss: 0.1112 - val_acc: 0.4285
Epoch 2/30
 - 5s - loss: 0.1060 - acc: 0.4700 - val_loss: 0.1022 - val_acc: 0.5087
Epoch 3/30
 - 5s - loss: 0.0995 - acc: 0.5293 - val_loss: 0.0975 - val_acc: 0.5306
Epoch 4/30
 - 5s - loss: 0.0958 - acc: 0.5467 - val_loss: 0.0943 - val_acc: 0.5563
Epoch 5/30
 - 5s - loss: 0.0936 - acc: 0.5546 - val_loss: 0.0927 - val_acc: 0.5551
Epoch 6/30
 - 5s - loss: 0.0920 - acc: 0.5610 - val_loss: 0.0911 - val_acc: 0.5679
Epoch 7/30
 - 5s - loss: 0.0909 - acc: 0.5669 - val_loss: 0.0903 - val_acc: 0.5699
Epoch 8/30
 - 5s - loss: 0.0901 - acc: 0.5718 - val_loss: 0.0897 - val_acc: 0.5799
Epoch 9/30
 - 5s - loss: 0.0894 - acc: 0.5750 - val_loss: 0.0891 - val_acc: 0.5764
Epoch 10/30
 - 5s - loss: 0.0889 - acc: 0.5771 - val_loss: 0.0885 - val_acc: 0.5816
Epoch 11/30
 - 5s - loss: 0.0885 - acc: 0.5798 - val_loss: 0.0882 - val_acc: 0.5840
Epoch 12/30
 - 5s - loss: 0.0880 - 

In [17]:
%%time
predictions = model.predict(test_data_transform)

CPU times: user 1.27 s, sys: 51 ms, total: 1.32 s
Wall time: 997 ms


In [20]:
predictions[0]*100

array([ 0.15347077,  3.1922393 ,  3.5636344 ,  5.342968  , 22.865559  ,
       64.882126  ], dtype=float32)

In [21]:
%%time
test_stars = np_utils.to_categorical(test_batch.stars)
score = model.evaluate(test_data_transform, test_stars, batch_size=128)

50000/50000 [==============================] - 1s 12us/step
CPU times: user 811 ms, sys: 52.3 ms, total: 863 ms
Wall time: 611 ms


In [22]:
score[1]*100

59.85400000190735